## Exercício final da unidade 4

## Leitura do dataset do kaggle

[Student Alcohol Consumption](https://www.kaggle.com/uciml/student-alcohol-consumption)

student-mat.csv

In [2]:
# Leitura de arquivo local
from google.colab import files
uploaded = files.upload()

Saving student-mat.csv to student-mat.csv


## Importação para o dataframe Pandas

In [40]:
# Uso do Pandas para criar o dataframe a partir do csv

import pandas as pd
df = pd.read_csv('student-mat.csv')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [0]:
# Extração das features e labels do dataframe

X = df.filter(items=['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','studytime','failures','activities','famrel','freetime','goout','Dalc','Walc','health','absences'])
y = df['G1']


In [0]:
# Feature Selection

# Coloca na matriz somente as features numéricas
X = df.filter(items=['age','Medu','Fedu','traveltime','studytime','failures','famrel','freetime','goout','Dalc','Walc','health','absences'])

# Adicionar uma feature de previsão com valor booleano
# Se estudante tirou nota >= 10 na G1, então foi aprovado
df['G1pass'] = df['G1'] >= 10
y = df['G1pass']

In [43]:
print(X.shape)
print(y.shape)

(395, 13)
(395,)


In [44]:
X.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences
0,18,4,4,2,2,0,4,3,4,1,1,3,6
1,17,1,1,1,2,0,5,3,3,1,1,3,4
2,15,1,1,1,2,3,4,3,2,2,3,3,10
3,15,4,2,1,3,0,3,2,2,1,1,5,2
4,16,3,3,1,2,0,4,3,2,1,2,5,4


## Engenharia de features

In [0]:
# Transforma os dataframes de features e label em formato numpy array
# Only the values in the DataFrame will be returned, the axes labels will be removed.

X = X.values
y = y.values

### Normalização das features

In [37]:
# Classe para normalizar os valores entre 0 e 1
from sklearn.preprocessing import MinMaxScaler

# Aplica transformação de MinMax nas features
X2 = MinMaxScaler().fit_transform(X)
X2

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


array([[0.42857143, 1.        , 1.        , ..., 0.        , 0.5       ,
        0.08      ],
       [0.28571429, 0.25      , 0.25      , ..., 0.        , 0.5       ,
        0.05333333],
       [0.        , 0.25      , 0.25      , ..., 0.5       , 0.5       ,
        0.13333333],
       ...,
       [0.85714286, 0.25      , 0.25      , ..., 0.5       , 0.5       ,
        0.04      ],
       [0.42857143, 0.75      , 0.5       , ..., 0.75      , 1.        ,
        0.        ],
       [0.57142857, 0.25      , 0.25      , ..., 0.5       , 1.        ,
        0.06666667]])

### Criação de features polinomiais

In [38]:
# Geração de features usando polinomios (x², x³)
from sklearn.preprocessing import PolynomialFeatures

# Criando as features polinomiais
model = PolynomialFeatures(degree=3, include_bias=False)
X2 = model.fit_transform(X)
print('Tamanho da tabela original: (Linhas, Colunas) {}'.format(X.shape))
print('Tamanho da tabela tratada: (Linhas, Colunas) {}'.format(X2.shape))
X2

Tamanho da tabela original: (Linhas, Colunas) (395, 13)
Tamanho da tabela tratada: (Linhas, Colunas) (395, 559)


array([[  18.,    4.,    4., ...,   54.,  108.,  216.],
       [  17.,    1.,    1., ...,   36.,   48.,   64.],
       [  15.,    1.,    1., ...,   90.,  300., 1000.],
       ...,
       [  21.,    1.,    1., ...,   27.,   27.,   27.],
       [  18.,    3.,    2., ...,    0.,    0.,    0.],
       [  19.,    1.,    1., ...,  125.,  125.,  125.]])

## Aprendizado supervisionado

### Divisão do dataframe em instâncias de treino e instâncias de teste

In [0]:
# Técnica do professor

train_percent = 0.7
train_index = int(len(X) * train_percent)
test_index = int(len(X) * (1 - train_percent))

X_train = X[:train_index]
y_train = y[:train_index]

X_test = X[-test_index:]
y_test = X[-test_index:]

In [50]:
# Técnica do curso DSA

from sklearn.model_selection import train_test_split

# Definindo a taxa de split
split_test_size = 0.30

# Criando dados de treino e de teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = split_test_size)

# Imprimindo os resultados
print("{0:0.2f}% nos dados de treino".format((len(X_treino)/len(df.index)) * 100))
print("{0:0.2f}% nos dados de teste".format((len(X_teste)/len(df.index)) * 100))

69.87% nos dados de treino
30.13% nos dados de teste


### Regressão

In [54]:
from sklearn.tree import DecisionTreeRegressor

# Criação e treino do modelo
model = DecisionTreeRegressor()
model_fit = model.fit(X_treino, y_treino)

# Verificação do erro médio absoluto em treino
from sklearn.metrics import mean_absolute_error
print('Mean Absolute Error:', mean_absolute_error(model_fit.predict(X_treino), y_treino))


Mean Absolute Error: 0.0036231884057971015


In [55]:
# Verificação do erro médio absoluto em teste
print('Mean Absolute Error:',
      mean_absolute_error(model_fit.predict(X_teste), y_teste))

Mean Absolute Error: 0.36554621848739494


### Classificação

In [56]:
from sklearn.tree import DecisionTreeClassifier

# Criação e treino do modelo
# model = DecisionTreeClassifier(splitter="random")
# model = DecisionTreeClassifier(presort=True)
model = DecisionTreeClassifier()
model_fit = model.fit(X_treino, y_treino)

# Verificação da acurácia em treino
from sklearn.metrics import accuracy_score
print('Model Accuracy:', accuracy_score(model_fit.predict(X_treino), y_treino))

Model Accuracy: 0.9963768115942029


In [57]:
# Verificação da acurácia em treino
print('Model Accuracy:', accuracy_score(model_fit.predict(X_teste), y_teste))


Model Accuracy: 0.6218487394957983
